In [27]:
!torchrun train.py --cfg-path train_configs/minigptv2_finetune.yaml

| distributed init (rank 0, world 1): env://
2024-04-16 21:39:35,394 [INFO] 
=====  Running Parameters    =====
2024-04-16 21:39:35,395 [INFO] {
    "amp": true,
    "device": "cuda",
    "dist_backend": "nccl",
    "dist_url": "env://",
    "distributed": true,
    "evaluate": false,
    "gpu": 0,
    "init_lr": 1e-05,
    "iters_per_epoch": 176948,
    "job_name": "pmcvqa_pretrain_w_bigger_mid_proj_w_trainable_visual_qv_w_sophia_w_dora",
    "lr_sched": "linear_warmup_cosine_lr",
    "max_epoch": 2,
    "min_lr": 1e-06,
    "num_workers": 6,
    "output_dir": "outputs",
    "rank": 0,
    "resume_ckpt_path": null,
    "seed": 42,
    "task": "image_text_pretrain",
    "train_splits": [
        "train"
    ],
    "wandb_log": true,
    "warmup_lr": 1e-06,
    "warmup_steps": 1000,
    "weight_decay": 0.05,
    "world_size": 1
}
2024-04-16 21:39:35,395 [INFO] 
======  Dataset Attributes  ======
2024-04-16 21:39:35,395 [INFO] 
======== pmcvqa =======
2024-04-16 21:39:35,395 [INFO] {
   

In [8]:
!python /ibex/user/sabbam0a/grad/MiniGPT-4/demo_v2.py --cfg-path /ibex/user/sabbam0a/grad/MiniGPT-4/eval_configs/minigptv2_eval.yaml --gpu-id 0

Initializing Chat
Loading checkpoint shards: 100%|██████████████████| 2/2 [00:02<00:00,  1.01s/it]
/home/sabbam0a/miniconda3/lib/python3.11/site-packages/peft/utils/other.py:141: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
trainable params: 33,554,432 || all params: 6,771,970,048 || trainable%: 0.49548996469513035
Position interpolate from 16x16 to 32x32
Load Minigpt-4-LLM Checkpoint: minigpt4/outputs/slake_finetune_w_mid_proj/checkpoint_0.pth
/home/sabbam0a/miniconda3/lib/python3.11/site-packages/gradio/layouts/column.py:55: UserWarning: 'scale' value should be an integer. Using 0.5 will cause issues.
  warnings.warn(
Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://ac9adf7e8049e549d3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://

In [ ]:
from bitsandbytes.cextension import CUDASetup

In [ ]:
lib = CUDASetup.get_instance().lib

In [ ]:
probs.last_hidden_state.shape

In [ ]:
(img_size=224,drop_path_rate=0.4,use_checkpoint=False,precision="fp16"):
    model = VisionTransformer(
        img_size=img_size,
        patch_size=14,
        use_mean_pooling=False,
        embed_dim=1408,
        depth=39,
        num_heads=1408//88,
        mlp_ratio=4.3637,
        qkv_bias=True,
        drop_path_rate=drop_path_rate,
        norm_layer=partial(nn.LayerNorm, eps=1e-6),
        use_checkpoint=use_checkpoint,
    )  

In [ ]:
import json
from langdetect import detect

In [ ]:
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        # In case langdetect cannot determine the language, consider it non-English
        return False

def filter_english_questions(json_file):
    with open(json_file, 'r', encoding='utf-8') as file:
        data = json.load(file)

    english_questions = [doc for doc in data if is_english(doc.get('question', ''))]

    return english_questions

In [ ]:
json_file = 'medvqa_dataset/Slake/Slake1.0/test.json'  # Replace with your JSON file path

english_questions = filter_english_questions(json_file)

# Save the filtered questions back to a new JSON file
with open('medvqa_dataset/Slake/Slake1.0/test_en.json', 'w', encoding='utf-8') as output_file:
    json.dump(english_questions, output_file, ensure_ascii=False, indent=2)

In [ ]:
import json

def merge_json_files(file1, file2, file3, output_file):
    with open(file1, 'r', encoding='utf-8') as f1:
        data1 = json.load(f1)

    with open(file2, 'r', encoding='utf-8') as f2:
        data2 = json.load(f2)

    with open(file3, 'r', encoding='utf-8') as f3:
        data3 = json.load(f3)

    # Merge the data from three files
    merged_data = data1 + data2 + data3

    # Save the merged data to a new JSON file
    with open(output_file, 'w', encoding='utf-8') as output:
        json.dump(merged_data, output, ensure_ascii=False, indent=2)


json_file1 = 'medvqa_dataset/Slake/Slake1.0/train_en.json'  # Replace with the path to your first JSON file
json_file2 = 'medvqa_dataset/Slake/Slake1.0/validate_en.json'  # Replace with the path to your second JSON file
json_file3 = 'medvqa_dataset/Slake/Slake1.0/test_en.json'  # Replace with the path to your third JSON file
output_json_file = 'medvqa_dataset/Slake/Slake1.0/all_annotations_en.json'  # Replace with the desired output file name

merge_json_files(json_file1, json_file2, json_file3, output_json_file)